## POS tagging using modified Viterbi - Aman Srivastava

### Data Preparation

In [1]:
import nltk
nltk.download('universal_tagset')

[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\amansrivasta\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [2]:
#Importing libraries
import nltk, re, pprint
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import pprint, time
import random
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize

In [3]:
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

In [4]:
# first few tagged sentences
print(nltk_data[:40])

[[('Pierre', 'NOUN'), ('Vinken', 'NOUN'), (',', '.'), ('61', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), (',', '.'), ('will', 'VERB'), ('join', 'VERB'), ('the', 'DET'), ('board', 'NOUN'), ('as', 'ADP'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('Nov.', 'NOUN'), ('29', 'NUM'), ('.', '.')], [('Mr.', 'NOUN'), ('Vinken', 'NOUN'), ('is', 'VERB'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Elsevier', 'NOUN'), ('N.V.', 'NOUN'), (',', '.'), ('the', 'DET'), ('Dutch', 'NOUN'), ('publishing', 'VERB'), ('group', 'NOUN'), ('.', '.')], [('Rudolph', 'NOUN'), ('Agnew', 'NOUN'), (',', '.'), ('55', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), ('and', 'CONJ'), ('former', 'ADJ'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Consolidated', 'NOUN'), ('Gold', 'NOUN'), ('Fields', 'NOUN'), ('PLC', 'NOUN'), (',', '.'), ('was', 'VERB'), ('named', 'VERB'), ('*-1', 'X'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('of', 'ADP'), ('this', 'DET'), ('British', 'ADJ'), ('industrial', 'ADJ'), ('

In [5]:
# Splitting into train and test
random.seed(1234)
train_set, test_set = train_test_split(nltk_data,test_size=0.05)

print(len(train_set))
print(len(test_set))
print(train_set[:40])

3718
196
[[('I', 'PRON'), ('was', 'VERB'), ('dumbfounded', 'ADJ'), (',', '.'), ("''", '.'), ('Mrs.', 'NOUN'), ('Ward', 'NOUN'), ('recalls', 'VERB'), ('*T*-1', 'X'), ('.', '.')], [('The', 'DET'), ('lower', 'ADJ'), ('figures', 'NOUN'), (',', '.'), ('the', 'DET'), ('spokesman', 'NOUN'), ('said', 'VERB'), ('0', 'X'), ('*T*-1', 'X'), (',', '.'), ('would', 'VERB'), ('stem', 'VERB'), ('from', 'ADP'), ('preferred', 'VERB'), ('shares', 'NOUN'), ('being', 'VERB'), ('converted', 'VERB'), ('*-88', 'X'), ('to', 'PRT'), ('common', 'ADJ'), ('stock', 'NOUN'), ('and', 'CONJ'), ('the', 'DET'), ('possibility', 'NOUN'), ('that', 'ADP'), ('Sea', 'NOUN'), ('Containers', 'NOUN'), ("'", 'PRT'), ('subsidiaries', 'NOUN'), ('might', 'VERB'), ('be', 'VERB'), ('required', 'VERB'), ('*-2', 'X'), ('to', 'PRT'), ('place', 'VERB'), ('their', 'PRON'), ('shares', 'NOUN'), ('in', 'ADP'), ('the', 'DET'), ('open', 'ADJ'), ('market', 'NOUN'), ('.', '.')], [('Other', 'ADJ'), ('financial', 'ADJ'), ('institutions', 'NOUN'), ('

In [6]:
# Getting list of tagged words
train_tagged_words = [tup for sent in train_set for tup in sent]
len(train_tagged_words)

95713

In [7]:
# tokens 
tokens = [pair[0] for pair in train_tagged_words]
tokens[:10]

['I', 'was', 'dumbfounded', ',', "''", 'Mrs.', 'Ward', 'recalls', '*T*-1', '.']

In [8]:
# vocabulary
V = set(tokens)
print(len(V))

12117


In [9]:
# number of tags
T = set([pair[1] for pair in train_tagged_words])
len(T)

12

In [10]:
print(T)

{'ADJ', 'CONJ', 'NOUN', 'VERB', 'NUM', '.', 'ADV', 'ADP', 'X', 'PRT', 'DET', 'PRON'}


### Emission Probabilities

In [11]:
# computing P(w/t) and storing in T x V matrix
t = len(T)
v = len(V)
w_given_t = np.zeros((t, v))

In [12]:
# compute word given tag: Emission Probability
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    count_w_given_tag = len(w_given_tag_list)
    
    return (count_w_given_tag, count_tag)

### Transition Probabilities

In [13]:
# compute tag given tag: tag2(t2) given tag1 (t1), i.e. Transition Probability

def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [14]:
# creating t x t transition matrix of tags
# each column is t2, each row is t1
# thus M(i, j) represents P(tj given ti)

tags_matrix = np.zeros((len(T), len(T)), dtype='float32')
for i, t1 in enumerate(list(T)):
    for j, t2 in enumerate(list(T)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]

In [15]:
# convert the matrix to a df for better readability
tags_df = pd.DataFrame(tags_matrix, columns = list(T), index=list(T))

In [16]:
tags_df.head()

,ADJ,CONJ,NOUN,VERB,NUM,.,ADV,ADP,X,PRT,DET,PRON
ADJ,0.066249,0.016974,0.698253,0.012030,0.021589,0.064436,0.004450,0.079103,0.020765,0.010547,0.004944,0.000659
CONJ,0.119555,0.000463,0.352178,0.156163,0.041242,0.035218,0.052363,0.051900,0.008341,0.005097,0.120019,0.057461
NOUN,0.012209,0.042567,0.263530,0.147017,0.009585,0.239805,0.017165,0.177776,0.028536,0.044098,0.012938,0.004774
VERB,0.065368,0.005279,0.110628,0.168388,0.023135,0.035090,0.081515,0.091530,0.218694,0.031364,0.133918,0.035090
NUM,0.033245,0.014122,0.350986,0.018241,0.185349,0.117976,0.002648,0.034716,0.210062,0.027655,0.003530,0.001471


In [17]:
len(train_tagged_words)

95713

### Build the vanilla Viterbi based POS tagger

In [18]:
# Viterbi Heuristic
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

####  Evaluating on Test Set for Viterbi

In [19]:
# list of tagged words
test_run_base = [tup for sent in test_set for tup in sent]

# list of untagged words
test_tagged_words = [tup[0] for sent in test_set for tup in sent]
test_set

[[('Mr.', 'NOUN'),
  ('Trump', 'NOUN'),
  ('withdrew', 'VERB'),
  ('a', 'DET'),
  ('$', '.'),
  ('120-a-share', 'ADJ'),
  ('*U*', 'X'),
  ('bid', 'NOUN'),
  ('last', 'ADJ'),
  ('month', 'NOUN'),
  ('.', '.')],
 [('Japan', 'NOUN'),
  ("'s", 'PRT'),
  ('Finance', 'NOUN'),
  ('Ministry', 'NOUN'),
  ('had', 'VERB'),
  ('set', 'VERB'),
  ('up', 'PRT'),
  ('mechanisms', 'NOUN'),
  ('0', 'X'),
  ('*-1', 'X'),
  ('to', 'PRT'),
  ('limit', 'VERB'),
  ('how', 'ADV'),
  ('far', 'ADJ'),
  ('futures', 'NOUN'),
  ('prices', 'NOUN'),
  ('could', 'VERB'),
  ('fall', 'VERB'),
  ('*T*-2', 'X'),
  ('in', 'ADP'),
  ('a', 'DET'),
  ('single', 'ADJ'),
  ('session', 'NOUN'),
  ('and', 'CONJ'),
  ('*-1', 'X'),
  ('to', 'PRT'),
  ('give', 'VERB'),
  ('market', 'NOUN'),
  ('operators', 'NOUN'),
  ('the', 'DET'),
  ('authority', 'NOUN'),
  ('*', 'X'),
  ('to', 'PRT'),
  ('suspend', 'VERB'),
  ('trading', 'NOUN'),
  ('in', 'ADP'),
  ('futures', 'NOUN'),
  ('at', 'ADP'),
  ('any', 'DET'),
  ('time', 'NOUN'),
  ('.

In [21]:
# tagging the test sentences using the vanilla veterbi
start = time.time()
tagged_seq_vanilla = Viterbi(test_tagged_words)
end = time.time()
difference = end-start
print("Time taken in seconds: ", difference)
print(tagged_seq_vanilla)

Time taken in seconds:  1004.010059595108
[('Mr.', 'NOUN'), ('Trump', 'NOUN'), ('withdrew', 'ADJ'), ('a', 'DET'), ('$', '.'), ('120-a-share', 'ADJ'), ('*U*', 'X'), ('bid', 'VERB'), ('last', 'ADJ'), ('month', 'NOUN'), ('.', '.'), ('Japan', 'NOUN'), ("'s", 'PRT'), ('Finance', 'NOUN'), ('Ministry', 'NOUN'), ('had', 'VERB'), ('set', 'VERB'), ('up', 'ADV'), ('mechanisms', 'ADJ'), ('0', 'X'), ('*-1', 'X'), ('to', 'PRT'), ('limit', 'NOUN'), ('how', 'ADV'), ('far', 'ADV'), ('futures', 'NOUN'), ('prices', 'NOUN'), ('could', 'VERB'), ('fall', 'VERB'), ('*T*-2', 'X'), ('in', 'ADP'), ('a', 'DET'), ('single', 'ADJ'), ('session', 'NOUN'), ('and', 'CONJ'), ('*-1', 'X'), ('to', 'PRT'), ('give', 'VERB'), ('market', 'NOUN'), ('operators', 'NOUN'), ('the', 'DET'), ('authority', 'NOUN'), ('*', 'X'), ('to', 'PRT'), ('suspend', 'VERB'), ('trading', 'NOUN'), ('in', 'ADP'), ('futures', 'NOUN'), ('at', 'ADP'), ('any', 'DET'), ('time', 'NOUN'), ('.', '.'), ('I', 'PRON'), ('have', 'VERB'), ('a', 'DET'), ('right'

In [22]:
# accuracy
check_vanilla = [i for i, j in zip(tagged_seq_vanilla, test_run_base) if i == j] 
accuracy_vanilla = len(check_vanilla)/len(tagged_seq_vanilla)
accuracy_vanilla

0.927664718920008

In [23]:
# Finiding the incorrectly tagged cases
incorrect_tagged_cases = [[test_run_base[i-1],j] for i, j in enumerate(zip(tagged_seq_vanilla, test_run_base)) if j[0]!=j[1]]

In [24]:
incorrect_tagged_cases

[[('Trump', 'NOUN'), (('withdrew', 'ADJ'), ('withdrew', 'VERB'))],
 [('*U*', 'X'), (('bid', 'VERB'), ('bid', 'NOUN'))],
 [('set', 'VERB'), (('up', 'ADV'), ('up', 'PRT'))],
 [('up', 'PRT'), (('mechanisms', 'ADJ'), ('mechanisms', 'NOUN'))],
 [('to', 'PRT'), (('limit', 'NOUN'), ('limit', 'VERB'))],
 [('how', 'ADV'), (('far', 'ADV'), ('far', 'ADJ'))],
 [('go', 'VERB'), (('there', 'DET'), ('there', 'ADV'))],
 [('and', 'CONJ'), (('laboriously', 'ADJ'), ('laboriously', 'ADV'))],
 [('but', 'CONJ'), (('no', 'DET'), ('no', 'ADV'))],
 [('longer', 'ADJ'), (('surreptitiously', 'ADJ'), ('surreptitiously', 'ADV'))],
 [('And', 'CONJ'), (('unlike', 'ADJ'), ('unlike', 'ADP'))],
 [('have', 'VERB'), (('difficulty', 'ADJ'), ('difficulty', 'NOUN'))],
 [('*-2', 'X'), (('shoring', 'ADJ'), ('shoring', 'VERB'))],
 [('market', 'NOUN'), (('crises', 'ADJ'), ('crises', 'NOUN'))],
 [('-LRB-', '.'), (('NYSE', 'ADJ'), ('NYSE', 'NOUN'))],
 [(';', '.'), (('Symbol', 'ADJ'), ('Symbol', 'NOUN'))],
 [(':', '.'), (('CSV', 'A

### Solve the problem of unknown words- Solution 1

In [25]:
# Modifying the vanilla veterbi. If the word in unknown then consider only the transition probability
def Viterbi_modified1(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            
            # if word in not in the training set then consider only the transition probability
            if word not in V:
                state_probability = transition_p    
            else:
                state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [26]:
# tagging the test sentences using the 1st modification
start = time.time()
tagged_seq_modified1 = Viterbi_modified1(test_tagged_words)
end = time.time()
difference = end-start
print("Time taken in seconds: ", difference)
print(tagged_seq_modified1)

Time taken in seconds:  924.1084666252136
[('Mr.', 'NOUN'), ('Trump', 'NOUN'), ('withdrew', 'NOUN'), ('a', 'DET'), ('$', '.'), ('120-a-share', 'NOUN'), ('*U*', 'X'), ('bid', 'VERB'), ('last', 'ADJ'), ('month', 'NOUN'), ('.', '.'), ('Japan', 'NOUN'), ("'s", 'PRT'), ('Finance', 'NOUN'), ('Ministry', 'NOUN'), ('had', 'VERB'), ('set', 'VERB'), ('up', 'ADV'), ('mechanisms', 'VERB'), ('0', 'X'), ('*-1', 'X'), ('to', 'PRT'), ('limit', 'NOUN'), ('how', 'ADV'), ('far', 'ADV'), ('futures', 'NOUN'), ('prices', 'NOUN'), ('could', 'VERB'), ('fall', 'VERB'), ('*T*-2', 'X'), ('in', 'ADP'), ('a', 'DET'), ('single', 'ADJ'), ('session', 'NOUN'), ('and', 'CONJ'), ('*-1', 'X'), ('to', 'PRT'), ('give', 'VERB'), ('market', 'NOUN'), ('operators', 'NOUN'), ('the', 'DET'), ('authority', 'NOUN'), ('*', 'X'), ('to', 'PRT'), ('suspend', 'VERB'), ('trading', 'NOUN'), ('in', 'ADP'), ('futures', 'NOUN'), ('at', 'ADP'), ('any', 'DET'), ('time', 'NOUN'), ('.', '.'), ('I', 'PRON'), ('have', 'VERB'), ('a', 'DET'), ('rig

#### Evaluating tagging accuracy

In [29]:
# accuracy
check_modified1 = [i for i, j in zip(tagged_seq_modified1, test_run_base) if i == j] 
accuracy_modified1 = len(check_modified1)/len(tagged_seq_modified1)
accuracy_modified1

0.9405601450735442

In [30]:
# Finiding the incorrectly tagged cases
incorrect_tagged_cases = [[test_run_base[i-1],j] for i, j in enumerate(zip(tagged_seq_modified1, test_run_base)) if j[0]!=j[1]]
incorrect_tagged_cases

[[('Trump', 'NOUN'), (('withdrew', 'NOUN'), ('withdrew', 'VERB'))],
 [('$', '.'), (('120-a-share', 'NOUN'), ('120-a-share', 'ADJ'))],
 [('*U*', 'X'), (('bid', 'VERB'), ('bid', 'NOUN'))],
 [('set', 'VERB'), (('up', 'ADV'), ('up', 'PRT'))],
 [('up', 'PRT'), (('mechanisms', 'VERB'), ('mechanisms', 'NOUN'))],
 [('to', 'PRT'), (('limit', 'NOUN'), ('limit', 'VERB'))],
 [('how', 'ADV'), (('far', 'ADV'), ('far', 'ADJ'))],
 [('go', 'VERB'), (('there', 'DET'), ('there', 'ADV'))],
 [('and', 'CONJ'), (('laboriously', 'NOUN'), ('laboriously', 'ADV'))],
 [('but', 'CONJ'), (('no', 'DET'), ('no', 'ADV'))],
 [('longer', 'ADJ'),
  (('surreptitiously', 'NOUN'), ('surreptitiously', 'ADV'))],
 [('And', 'CONJ'), (('unlike', 'ADJ'), ('unlike', 'ADP'))],
 [('have', 'VERB'), (('difficulty', 'X'), ('difficulty', 'NOUN'))],
 [('shoring', 'VERB'), (('up', 'ADV'), ('up', 'PRT'))],
 [('his', 'PRON'), (('return', 'VERB'), ('return', 'NOUN'))],
 [("''", '.'), (('Municipal', 'NOUN'), ('Municipal', 'ADJ'))],
 [('took',

### Solve the problem of unknown words - Solution 2

In [31]:
# importing PorterStemmer
from nltk.stem.porter import PorterStemmer

In [32]:
# Creating PorterStemmer object
stemmer = PorterStemmer()

In [33]:
# Creating a dictionary of stemmized words with the original POS tags
stemmized_x = {}
for item in train_tagged_words:
    stemmized_x[stemmer.stem(item[0])] = item[1]

In [34]:
# method to return pos tags based on different rules
def custom_pos_tagger(word):
    stemmed_word = stemmer.stem(word)
    # checking if the input word from the test data is present on the stemmized_x dictinary
    if stemmed_word in stemmized_x:
        return stemmized_x[stemmed_word]
    # checking if the input word from the test data is present in the lower_train dictionary
    elif word.lower() in lower_train:
        return lower_train[word.lower()]
    else:
        return morphological_rule_based(word)

In [35]:
# Creating a dictinory of the lower cases of the words from train set with their POS tags
lower_train = {}
for item in train_tagged_words:
    lower_train[item[0].lower()] = item[1]

In [36]:
# Defining a method for morphological rule based POS tagging
def morphological_rule_based(word):
    suffix_1= ["ing", "ed", "es"]
    if word.istitle():
        return "NOUN"
    elif word.endswith("ing") or word.endswith("ed"):
        return "VERB"
    elif word.lower() in ['a', 'an', 'the', 'A', 'An', 'The']:
        return "DET"
    else:
        return None

In [37]:
# If emission probability is zero we consider only the trasition probaility
def Viterbi_modified2(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        
        if word not in V:
            state_max = custom_pos_tagger(word)
            # checking if our custom pos tagger worked or not. If it didnt work it will return 'None'
            if state_max is None:
                p = []
                for tag in T:
                    if key == 0:
                        transition_p = tags_df.loc['.', tag]
                    else:
                        transition_p = tags_df.loc[state[-1], tag]
                    
                    state_probability = transition_p    
                    p.append(state_probability)
                
                pmax = max(p)
                # getting state for which probability is maximum
                state_max = T[p.index(pmax)] 
            
        else:
            p = [] 
            for tag in T:
                if key == 0:
                    transition_p = tags_df.loc['.', tag]
                else:
                    transition_p = tags_df.loc[state[-1], tag]

                # compute emission and state probabilities
                emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
                state_probability = emission_p * transition_p    
                p.append(state_probability)

            pmax = max(p)
            # getting state for which probability is maximum
            state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

#### Evaluating tagging accuracy

In [38]:
# tagging the test sentences using 2nd modification
start = time.time()
tagged_seq_modified2 = Viterbi_modified2(test_tagged_words)
end = time.time()
difference = end-start
print("Time taken in seconds: ", difference)
print(tagged_seq_modified2)

Time taken in seconds:  819.0802927017212
[('Mr.', 'NOUN'), ('Trump', 'NOUN'), ('withdrew', 'NOUN'), ('a', 'DET'), ('$', '.'), ('120-a-share', 'NOUN'), ('*U*', 'X'), ('bid', 'VERB'), ('last', 'ADJ'), ('month', 'NOUN'), ('.', '.'), ('Japan', 'NOUN'), ("'s", 'PRT'), ('Finance', 'NOUN'), ('Ministry', 'NOUN'), ('had', 'VERB'), ('set', 'VERB'), ('up', 'ADV'), ('mechanisms', 'NOUN'), ('0', 'X'), ('*-1', 'X'), ('to', 'PRT'), ('limit', 'NOUN'), ('how', 'ADV'), ('far', 'ADV'), ('futures', 'NOUN'), ('prices', 'NOUN'), ('could', 'VERB'), ('fall', 'VERB'), ('*T*-2', 'X'), ('in', 'ADP'), ('a', 'DET'), ('single', 'ADJ'), ('session', 'NOUN'), ('and', 'CONJ'), ('*-1', 'X'), ('to', 'PRT'), ('give', 'VERB'), ('market', 'NOUN'), ('operators', 'NOUN'), ('the', 'DET'), ('authority', 'NOUN'), ('*', 'X'), ('to', 'PRT'), ('suspend', 'VERB'), ('trading', 'NOUN'), ('in', 'ADP'), ('futures', 'NOUN'), ('at', 'ADP'), ('any', 'DET'), ('time', 'NOUN'), ('.', '.'), ('I', 'PRON'), ('have', 'VERB'), ('a', 'DET'), ('rig

In [40]:
# accuracy
check_modified2 = [i for i, j in zip(tagged_seq_modified2, test_run_base) if i == j] 
accuracy_modified2 = len(check_modified2)/len(tagged_seq_modified2)
accuracy_modified2

0.9443884747128752

### Compare the tagging accuracies of the modifications with the vanilla Viterbi algorithm

In [41]:
# Accuracy with vanilla viterbi
accuracy_vanilla

0.927664718920008

In [42]:
# Accuracy with vanilla viterbi with modification 1
accuracy_modified1

0.9405601450735442

In [43]:
# Accuracy with vanilla viterbi with modification 2
accuracy_modified2

0.9443884747128752

### List down cases which were incorrectly tagged by original POS tagger and got corrected by your modifications

In [45]:
# Using the sample test sentense given in problem
sample_test_sentenses = ["Android is a mobile operating system developed by Google.",
                         "Android has been the best-selling OS worldwide on smartphones since 2011 and on tablets since 2013.",
                         "Google and Twitter made a deal in 2015 that gave Google access to Twitter's firehose.",
                         "Twitter is an online news and social networking service on which users post and interact with messages known as tweets.",
                         "Before entering politics, Donald Trump was a domineering businessman and a television personality.",
                         "The 2018 FIFA World Cup is the 21st FIFA World Cup, an international football tournament contested once every four years.",
                         "This is the first World Cup to be held in Eastern Europe and the 11th time that it has been held in Europe.",
                         "Show me the cheapest round trips from Dallas to Atlanta",
                         "I would like to see flights from Denver to Philadelphia.",
                         "Show me the price of the flights leaving Atlanta at about 3 in the afternoon and arriving in San Francisco.",
                         "NASA invited social media users to experience the launch of ICESAT-2 Satellite."]
print(sample_test_sentenses)

['Android is a mobile operating system developed by Google.', 'Android has been the best-selling OS worldwide on smartphones since 2011 and on tablets since 2013.', "Google and Twitter made a deal in 2015 that gave Google access to Twitter's firehose.", 'Twitter is an online news and social networking service on which users post and interact with messages known as tweets.', 'Before entering politics, Donald Trump was a domineering businessman and a television personality.', 'The 2018 FIFA World Cup is the 21st FIFA World Cup, an international football tournament contested once every four years.', 'This is the first World Cup to be held in Eastern Europe and the 11th time that it has been held in Europe.', 'Show me the cheapest round trips from Dallas to Atlanta', 'I would like to see flights from Denver to Philadelphia.', 'Show me the price of the flights leaving Atlanta at about 3 in the afternoon and arriving in San Francisco.', 'NASA invited social media users to experience the laun

In [46]:
nltk.download('punkt')
sample_words = []
# # tokenize into words
tokenized_sents = [word_tokenize(i) for i in sample_test_sentenses]
for i in tokenized_sents:
    for k in i:
        sample_words.append(k)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\amansrivasta\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [47]:
# tagging the sample sentences with vanilla viterbi
start = time.time()
tagged_seq = Viterbi(sample_words)
end = time.time()
difference = end-start
print("Time taken in seconds: ", difference)
print(tagged_seq)

Time taken in seconds:  35.65602254867554
[('Android', 'ADJ'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'ADJ'), ('.', '.'), ('Android', 'ADJ'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'ADJ'), ('worldwide', 'ADJ'), ('on', 'ADP'), ('smartphones', 'ADJ'), ('since', 'ADP'), ('2011', 'ADJ'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'ADJ'), ('.', '.'), ('Google', 'ADJ'), ('and', 'CONJ'), ('Twitter', 'ADJ'), ('made', 'VERB'), ('a', 'DET'), ('deal', 'NOUN'), ('in', 'ADP'), ('2015', 'ADJ'), ('that', 'ADP'), ('gave', 'VERB'), ('Google', 'ADJ'), ('access', 'NOUN'), ('to', 'PRT'), ('Twitter', 'ADJ'), ("'s", 'PRT'), ('firehose', 'ADJ'), ('.', '.'), ('Twitter', 'ADJ'), ('is', 'VERB'), ('an', 'DET'), ('online', 'ADJ'), ('news', 'NOUN'), ('and', 'CONJ'), ('social', 'ADJ'), ('networking', 'NOUN'), ('service', 'NOUN'), ('on'

In [48]:
# tagging the sample sentences with 2nd modified viterbi algo
start = time.time()
tagged_seq = Viterbi_modified2(sample_words)
end = time.time()
difference = end-start
print("Time taken in seconds: ", difference)
print(tagged_seq)

Time taken in seconds:  27.105536222457886
[('Android', 'NOUN'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'NOUN'), ('.', '.'), ('Android', 'NOUN'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'NOUN'), ('worldwide', 'NOUN'), ('on', 'ADP'), ('smartphones', 'DET'), ('since', 'ADP'), ('2011', 'DET'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'DET'), ('.', '.'), ('Google', 'NOUN'), ('and', 'CONJ'), ('Twitter', 'NOUN'), ('made', 'VERB'), ('a', 'DET'), ('deal', 'NOUN'), ('in', 'ADP'), ('2015', 'DET'), ('that', 'ADP'), ('gave', 'VERB'), ('Google', 'NOUN'), ('access', 'NOUN'), ('to', 'PRT'), ('Twitter', 'NOUN'), ("'s", 'PRT'), ('firehose', 'VERB'), ('.', '.'), ('Twitter', 'NOUN'), ('is', 'VERB'), ('an', 'DET'), ('online', 'NOUN'), ('news', 'NOUN'), ('and', 'CONJ'), ('social', 'ADJ'), ('networking', 'NOUN'), ('service', '

## We can clearly see that Android, Google and Twitter were being tagged as 'CONJ' with the vanilla veterbi algo and with my modifications they are not correctly being tagged as 'NOUN'.